# Tratando dados de transações

## Assistir course de time series do kaggle

## Etapas

- Ler os dados V
- Analise dos dados V
- Remover colunas menos importantes, filtrar soja V
- Verificar e tratar valores nulos V
- Verificar e tratar outliers
- Relação de data e seller id
- Tratamento para unificar data e criar features unificadas
- Separar por seller id
- Expandir datas
- Tratamento de valores nulos
- Criar features com os dados do seller id

## Features interessantes para o seller id

Features que já possuimos:
- Data
- Price por data
- Amount por data
Features que iremos criar:
- Preço medio diario
- Quantidade media diaria
- Maior preço vendido
- Menor preço vendido
- Maior quantidade vendida
- Menor quantidade vendida
- Quantidade de transações por dia
- Se a quantidade do menor valor é maior que a quantidade do maior valor(Se vendeu mais com preço menor)
- Dia da semana
- Se transacionou ou não(0 ou 1)

## Lendo dados

In [1]:
import pandas as pd

In [2]:
url = 'C:/workspace/Desafio Grão Direto IA/Data/transações-desafio.xlsx'

transacoes = pd.read_excel(url)
transacoes

,Date,Time,Company,Seller ID,Buyer ID,Price,Amount,Product,origin_city,origin_state
0,2024-11-04,14:21:17,Polaris,100000060,200000027,124.447569,2290.909287,Soja,Porto Velho,RO
1,2024-11-04,12:11:56,Polaris,100002271,200000180,119.524125,2092.071936,Soja,Uberlândia,MG
2,2024-11-04,12:20:33,Polaris,100002332,200000199,121.641336,25427.175976,Soja,Terra Nova do Norte,MT
3,2024-11-04,11:32:09,Lunarix,100002035,200000020,119.025800,5074.951735,Soja,Barcarena,PA
4,2024-11-04,15:16:05,Polaris,100002339,200000208,114.041989,9691.241875,Soja,Tasso Fragoso,MA
...,...,...,...,...,...,...,...,...,...,...
9508,2024-01-03,12:15:56,Lunarix,100000094,200000011,107.246483,2009.361816,Soja,Porto Velho,RO
9509,2024-01-03,13:26:05,Lunarix,100000314,200000009,112.163921,962.326195,Soja,Boa Vista,RR
9510,2024-01-03,11:37:41,Polaris,100000865,200000027,137.791300,107351.263706,Soja,Rondonópolis,MT
9511,2024-01-03,14:24:58,Polaris,100000866,200000133,127.947271,1353.466033,Soja,Rondonópolis,MT


## Filtrando por soja

In [3]:
# transacoes_filtradas = transacoes[transacoes['Product'] == 'Soja']

## Dropando colunas

In [4]:
transacoes_filtradas = transacoes[['Date','Seller ID']]
transacoes_filtradas

,Date,Seller ID
0,2024-11-04,100000060
1,2024-11-04,100002271
2,2024-11-04,100002332
3,2024-11-04,100002035
4,2024-11-04,100002339
...,...,...
9508,2024-01-03,100000094
9509,2024-01-03,100000314
9510,2024-01-03,100000865
9511,2024-01-03,100000866


## Tratando valores nulos

In [5]:
transacoes_filtradas.isnull().sum()

Date         0
Seller ID    0
dtype: int64

## Expandindo a série temporal do Seller ID

In [6]:
date_range = pd.date_range(start='2024-01-30',end='2024-11-05',freq='d')
df_dates = pd.DataFrame(date_range, columns=['Date'])
df_dates

,Date
0,2024-01-30
1,2024-01-31
2,2024-02-01
3,2024-02-02
4,2024-02-03
...,...
276,2024-11-01
277,2024-11-02
278,2024-11-03
279,2024-11-04


# Separando por seller id

In [7]:
transacoes_filtradas.shape

(9513, 2)

In [8]:
transacoes_filtradas = transacoes_filtradas.drop_duplicates(subset=['Date','Seller ID']).reset_index(drop=True)
transacoes_filtradas.shape

(7654, 2)

In [9]:
unique_seller_ids = transacoes_filtradas['Seller ID'].unique()

all_seller_data = []

for seller in unique_seller_ids:
    seller_data = transacoes_filtradas[transacoes_filtradas['Seller ID'] == seller]
    seller_data = seller_data.sort_values(by='Date', ascending=True)
    seller_data['Transactioned'] = 1
    seller_data = pd.merge(df_dates, seller_data, on='Date', how='left')
    seller_data['Transactioned'] = seller_data['Transactioned'].fillna(0).astype('Int64')
    seller_data['Seller ID'] = seller
    all_seller_data.append(seller_data[['Date', 'Seller ID', 'Transactioned']])

transacoes_sellerid = pd.concat(all_seller_data, ignore_index=True)
transacoes_sellerid

,Date,Seller ID,Transactioned
0,2024-01-30,100000060,0
1,2024-01-31,100000060,0
2,2024-02-01,100000060,0
3,2024-02-02,100000060,0
4,2024-02-03,100000060,0
...,...,...,...
682263,2024-11-01,100000314,0
682264,2024-11-02,100000314,0
682265,2024-11-03,100000314,0
682266,2024-11-04,100000314,0


# Criando features

In [10]:
dias = []

for seller_id in transacoes_sellerid['Seller ID'].unique():
    seller_data = transacoes_sellerid[transacoes_sellerid['Seller ID'] == seller_id]
    cont = -1
    dias_seller = []
    for transaction in seller_data['Transactioned']:
        if transaction == 1:
            cont = 0
        elif cont >= 0:
            cont += 1
        dias_seller.append(cont)
    dias.extend(dias_seller)

transacoes_sellerid['Last Transaction'] = dias
transacoes_sellerid['Last Transaction'] = transacoes_sellerid['Last Transaction'].replace(-1, 0)
transacoes_sellerid
    

,Date,Seller ID,Transactioned,Last Transaction
0,2024-01-30,100000060,0,0
1,2024-01-31,100000060,0,0
2,2024-02-01,100000060,0,0
3,2024-02-02,100000060,0,0
4,2024-02-03,100000060,0,0
...,...,...,...,...
682263,2024-11-01,100000314,0,0
682264,2024-11-02,100000314,0,0
682265,2024-11-03,100000314,0,0
682266,2024-11-04,100000314,0,0


In [11]:
transacoes_sellerid['Last Week'] = (transacoes_sellerid.groupby('Seller ID')['Transactioned']
                                    .transform(lambda x: x.rolling(window=7,min_periods=1).sum())
                                    .apply(lambda x: 1 if x >0 else 0 )
                                    .astype('Int64'))
transacoes_sellerid['Last Month'] = (transacoes_sellerid.groupby('Seller ID')['Transactioned']
                                    .transform(lambda x: x.rolling(window=30,min_periods=1).sum())
                                    .apply(lambda x: 1 if x >0 else 0 )
                                    .astype('Int64'))
transacoes_sellerid

,Date,Seller ID,Transactioned,Last Transaction,Last Week,Last Month
0,2024-01-30,100000060,0,0,0,0
1,2024-01-31,100000060,0,0,0,0
2,2024-02-01,100000060,0,0,0,0
3,2024-02-02,100000060,0,0,0,0
4,2024-02-03,100000060,0,0,0,0
...,...,...,...,...,...,...
682263,2024-11-01,100000314,0,0,0,0
682264,2024-11-02,100000314,0,0,0,0
682265,2024-11-03,100000314,0,0,0,0
682266,2024-11-04,100000314,0,0,0,0


In [12]:
transacoes_sellerid['Day of Week'] = transacoes_sellerid['Date'].dt.day_of_week
transacoes_sellerid = transacoes_sellerid[['Date','Seller ID','Last Transaction','Last Week','Last Month','Day of Week','Transactioned']]
transacoes_sellerid

,Date,Seller ID,Last Transaction,Last Week,Last Month,Day of Week,Transactioned
0,2024-01-30,100000060,0,0,0,1,0
1,2024-01-31,100000060,0,0,0,2,0
2,2024-02-01,100000060,0,0,0,3,0
3,2024-02-02,100000060,0,0,0,4,0
4,2024-02-03,100000060,0,0,0,5,0
...,...,...,...,...,...,...,...
682263,2024-11-01,100000314,0,0,0,4,0
682264,2024-11-02,100000314,0,0,0,5,0
682265,2024-11-03,100000314,0,0,0,6,0
682266,2024-11-04,100000314,0,0,0,0,0


# Salvando transações em um excel

In [13]:
transacoes_sellerid.to_excel("C:/workspace/Desafio Grão Direto IA/Data/transações-desafio-tratado.xlsx", index=False)

In [14]:
path_mercado = 'C:/workspace/Desafio Grão Direto IA/Data/mercado-desafio-tratado.xlsx'
path_transacoes = 'C:/workspace/Desafio Grão Direto IA/Data/transações-desafio-tratado.xlsx'

mercado = pd.read_excel(path_mercado)
transacoes =  pd.read_excel(path_transacoes)

In [15]:
seller_transactioned_sum = transacoes.groupby('Seller ID')['Transactioned'].sum()
seller_ids = seller_transactioned_sum[seller_transactioned_sum > 2].index
transacoes = transacoes[transacoes['Seller ID'].isin(seller_ids)]

In [16]:
model_df = pd.merge(transacoes,mercado,on='Date',how='left')
model_df.to_excel('C:/workspace/Desafio Grão Direto IA/Data/model_df.xlsx',index=False)